# Debug JupyterServerApp Import Error

This notebook demonstrates how to resolve the `name 'JupyterServerApp' is not defined` error and properly import Jupyter server components for JupyterLab extensions.

## Overview

The error typically occurs when:
1. Incorrect import statements for Jupyter server components
2. Version mismatches between jupyter-server and extension code
3. Missing or incorrect entry point configurations
4. Runtime import issues in server extensions

## 1. Import Required Modules

First, let's import the necessary libraries and modules for working with Jupyter server components.

In [1]:
import sys
import os
import subprocess
import importlib
import pkg_resources
from importlib import util

print("Python version:", sys.version)
print("Current working directory:", os.getcwd())

Python version: 3.12.3 (main, Feb  4 2025, 14:48:35) [GCC 13.3.0]
Current working directory: /home/bdx/allcode/github/vantagecompute/jfl


/tmp/ipykernel_1360276/331945127.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## 2. Check Jupyter Server Installation

Let's verify that jupyter-server is properly installed and check the version to ensure compatibility.

In [2]:
# Check if jupyter-server is installed
try:
    import jupyter_server
    print(f"✅ jupyter-server version: {jupyter_server.__version__}")
except ImportError as e:
    print(f"❌ jupyter-server not found: {e}")

# Check related packages
packages_to_check = [
    'jupyterlab',
    'jupyter_client', 
    'tornado',
    'traitlets'
]

for package in packages_to_check:
    try:
        version = pkg_resources.get_distribution(package).version
        print(f"✅ {package} version: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"❌ {package} not found")

# Check our Firefox launcher extension
try:
    import jupyterlab_firefox_launcher
    print(f"✅ jupyterlab-firefox-launcher version: {jupyterlab_firefox_launcher.__version__}")
except ImportError as e:
    print(f"❌ jupyterlab-firefox-launcher not found: {e}")
except AttributeError:
    print("✅ jupyterlab-firefox-launcher imported but no version info")

✅ jupyter-server version: 2.16.0
✅ jupyterlab version: 4.4.5
✅ jupyter_client version: 8.6.3
✅ tornado version: 6.5.1
✅ traitlets version: 5.14.3
✅ jupyterlab-firefox-launcher version: 0.1.0


## 3. Demonstrate the Error

Let's show how the NameError occurs when trying to use JupyterServerApp without proper imports.

In [ ]:
# This would cause the error: NameError: name 'JupyterServerApp' is not defined
try:
    # Simulating the problematic code without import
    def bad_function(serverapp):
        # This line would fail if JupyterServerApp wasn't imported
        # but used in type annotation: def function(serverapp: JupyterServerApp)
        return f"Server app type: {type(serverapp)}"
    
    print("✅ Function defined successfully (no type annotation issue here)")
    
except NameError as e:
    print(f"❌ NameError occurred: {e}")

# Show what happens when we try to reference JupyterServerApp without importing
try:
    # This will definitely fail
    app_class = JupyterServerApp
    print(f"App class: {app_class}")
except NameError as e:
    print(f"❌ Expected NameError: {e}")
    print("This is the error we need to fix in our server extension!")

## 4. Import JupyterServerApp Correctly

Now let's demonstrate the correct ways to import JupyterServerApp and handle potential import issues.

In [ ]:
# Method 1: Direct import from jupyter_server.application
try:
    from jupyter_server.application import JupyterServerApp
    print("✅ Method 1: Successfully imported JupyterServerApp from jupyter_server.application")
    print(f"   JupyterServerApp class: {JupyterServerApp}")
    print(f"   Module: {JupyterServerApp.__module__}")
except ImportError as e:
    print(f"❌ Method 1 failed: {e}")

# Method 2: Alternative import path (if available)
try:
    from jupyter_server.serverapp import JupyterServerApp as ServerApp
    print("✅ Method 2: Successfully imported from jupyter_server.serverapp")
    print(f"   ServerApp class: {ServerApp}")
except ImportError as e:
    print(f"❌ Method 2 failed: {e}")

# Method 3: Defensive import with fallback
try:
    try:
        from jupyter_server.application import JupyterServerApp
        print("✅ Method 3: Primary import successful")
    except ImportError:
        # Fallback for older versions
        from jupyter_server.serverapp import JupyterServerApp
        print("✅ Method 3: Fallback import successful")
        
    # Test that we can use it
    def safe_function(serverapp):
        """This function can now safely reference JupyterServerApp"""
        if isinstance(serverapp, JupyterServerApp):
            return f"✅ Server app is valid: {type(serverapp).__name__}"
        else:
            return f"⚠️ Server app type: {type(serverapp).__name__}"
    
    print("✅ Safe function defined with proper import")
    
except ImportError as e:
    print(f"❌ All import methods failed: {e}")

## 5. Alternative Import Methods

Let's explore robust import patterns for server extensions that work across different Jupyter versions.

In [ ]:
# Pattern 1: Runtime import (no type annotation issues)
def create_extension_loader():
    """Extension loader that imports JupyterServerApp at runtime"""
    try:
        from jupyter_server.application import JupyterServerApp
        return JupyterServerApp
    except ImportError:
        print("Could not import JupyterServerApp")
        return None

ServerAppClass = create_extension_loader()
if ServerAppClass:
    print(f"✅ Runtime import successful: {ServerAppClass.__name__}")

# Pattern 2: Module-level import with error handling
def safe_import():
    """Safe import that handles missing dependencies"""
    try:
        import jupyter_server.application
        return jupyter_server.application.JupyterServerApp
    except (ImportError, AttributeError) as e:
        print(f"Import failed: {e}")
        return object  # Fallback to basic object class

SafeServerApp = safe_import()
print(f"✅ Safe import result: {SafeServerApp}")

# Pattern 3: Extension function without type annotations
def load_jupyter_server_extension(serverapp):
    """
    Extension loader that doesn't use type annotations
    This avoids import-time errors
    """
    # We can check the type at runtime instead
    app_type = type(serverapp).__name__
    print(f"✅ Extension loaded for server app: {app_type}")
    
    # Register handlers here
    web_app = serverapp.web_app
    print(f"✅ Web app available: {type(web_app).__name__}")
    
    return True

# Test the pattern
print("✅ Extension function defined without import errors")

## 6. Verify JupyterLab Extension Configuration

Let's test that our Firefox launcher extension entry points and server configuration work correctly with the proper imports.

In [ ]:
# Test our extension's entry points
try:
    from jupyterlab_firefox_launcher import _jupyter_server_extension_points
    
    extension_points = _jupyter_server_extension_points()
    print("✅ Extension points function works:")
    for point in extension_points:
        print(f"   - Module: {point.get('module', 'N/A')}")
        print(f"   - App: {point.get('app', 'N/A')}")
        
except ImportError as e:
    print(f"❌ Extension import failed: {e}")
except Exception as e:
    print(f"❌ Extension points error: {e}")

# Test server extension loading
try:
    from jupyterlab_firefox_launcher.server_extension import _load_jupyter_server_extension
    print("✅ Server extension loader imported successfully")
    print(f"   Function: {_load_jupyter_server_extension}")
    
    # Test if it can be called (we'll use a mock object)
    class MockServerApp:
        def __init__(self):
            self.log = self
            self.web_app = MockWebApp()
            
        def info(self, msg):
            print(f"MockServer: {msg}")
            
        def warning(self, msg):
            print(f"MockServer WARNING: {msg}")
            
        def error(self, msg):
            print(f"MockServer ERROR: {msg}")
    
    class MockWebApp:
        def __init__(self):
            self.settings = {"base_url": "/"}
            
        def add_handlers(self, pattern, handlers):
            print(f"✅ Mock handlers added: {len(handlers)} handlers")
    
    # Test the extension loader
    mock_app = MockServerApp()
    _load_jupyter_server_extension(mock_app)
    print("✅ Extension loaded successfully with mock server app")
    
except ImportError as e:
    print(f"❌ Server extension import failed: {e}")
except Exception as e:
    print(f"❌ Server extension test failed: {e}")

# Test handler imports
try:
    from jupyterlab_firefox_launcher.firefox_handler import (
        FirefoxLauncherHandler, 
        FirefoxCleanupHandler,
        get_server_proxy_config
    )
    print("✅ Handler imports successful:")
    print(f"   - FirefoxLauncherHandler: {FirefoxLauncherHandler}")
    print(f"   - FirefoxCleanupHandler: {FirefoxCleanupHandler}")
    print(f"   - get_server_proxy_config: {get_server_proxy_config}")
    
    # Test proxy config
    proxy_config = get_server_proxy_config()
    print(f"✅ Proxy config generated: {len(proxy_config)} entries")
    
except ImportError as e:
    print(f"❌ Handler import failed: {e}")
except Exception as e:
    print(f"❌ Handler test failed: {e}")

✅ Extension points function works:
   - Module: jupyterlab_firefox_launcher.server_extension
   - App: _load_jupyter_server_extension
✅ Server extension loader imported successfully
   Function: <function _load_jupyter_server_extension at 0x7aae2c7f65c0>
MockServer: Loading jupyterlab_firefox_launcher extension
✅ Mock handlers added: 2 handlers
MockServer: jupyter-server-proxy detected, registering Firefox proxy
MockServer ERROR: ❌ Error configuring proxy: AttributeError: 'MockServerApp' object has no attribute 'debug'
MockServer: Firefox launcher will use direct API endpoints instead
MockServer: jupyterlab_firefox_launcher extension loaded successfully
MockServer: Firefox launcher API available at: /firefox-launcher/api/firefox
✅ Extension loaded successfully with mock server app
✅ Handler imports successful:
   - FirefoxLauncherHandler: <class 'jupyterlab_firefox_launcher.firefox_handler.FirefoxLauncherHandler'>
   - FirefoxCleanupHandler: <class 'jupyterlab_firefox_launcher.firefox_

: 

## Summary and Resolution

### The Fix Applied

We resolved the `name 'JupyterServerApp' is not defined` error by:

1. **Removing the problematic type annotation** from the `_load_jupyter_server_extension` function
2. **Updating the extension entry point** to include the proper `app` field
3. **Using runtime type checking** instead of compile-time type annotations

### Key Changes Made

```python
# Before (causing error):
def _load_jupyter_server_extension(serverapp: JupyterServerApp):
    pass

# After (working):
def _load_jupyter_server_extension(serverapp):
    """Extension loader without type annotation to avoid import errors"""
    pass
```

### Entry Point Configuration

```toml
[project.entry-points."jupyter_server.extension_points"]
jupyterlab_firefox_launcher = "jupyterlab_firefox_launcher:_jupyter_server_extension_points"
```

The extension should now load properly without the import error. The 404 error on the API endpoint should be resolved once JupyterLab is restarted with the corrected extension.